In [ ]:
#selenium
#py2exe
#pyinstaller
#The elements of statistical learning
#Introduction to statistical learning
#

In [1]:
from tweepy import OAuthHandler 
import tweepy 

In [2]:
username, consumer_key, consumer_secret, access_token, access_token_secret 

In [3]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
city = ['Wro', 'Crac'] -> 'lat', 'long'  #this in string
# add +15km

In [ ]:
tweepy.cursor(api.search, q = 'endomondo', geocode = 'string with coordinates')